In [22]:
%run training_functions.ipynb

['train', 'validation', 'test']
['train', 'validation', 'test']
In, Out, get_ipython, exit, quit, open, accelerate, datasets, evaluate, math, np, peft, pickle, pytest, ipytest, pd, transformers, torch, load_dataset, load_dataset_builder, get_dataset_split_names, get_dataset_config_names, LoftQConfig, LoraConfig, get_peft_model, AutoModelForCausalLM, AutoTokenizer, pipeline, TrainingArguments, Trainer, SFTTrainer, SFTConfig, @py_builtins, @pytest_ar, model_from_pkl, ds_gst1_train, ds_gst1_test, ds_gst2_train, ds_gst2_test, search_data, search_with_strings, map_data, print_trainable_parameters, metric, comp_metrics_output, compute_metrics, make_trainer, get_dataframe, get_training_output, mis_model, mis_token, pipe, messages, output, mis_epoch, mis_token_epoch, continued_convo, model, answer, answer2, answer3, answer4, 

## Dataset value example 
{'record': 'NZ_LIXJ01000001.1', 'text': 'CGACTCCGCACAGGCCCTAGGCGTGGTCCACGCATCGAGCGCGTGCGCCGTGGCATACACCCGGGCCACCCGTACGACCCTTCCGGCCGCCCCGTGTGCGCCCTCGTCGCCGAGTCCGCTCCGGCGCGCGCGGGCGCACTGCTGTTCCGCGCGGTGTCATGCGCTCCTCCCGTCCCGACATGCGCCTTTTTGCAGGGGAGAGGGAGGGGTAGGGTCGCGCGTACGCCTAGCACGGGGGAACGGTCTTGGGGGACGCAGTGGACGTACGGGTGGCGGCGATCGCCAGTCTCACACCGCTCGAAGAACTCGACAGCGATCCGTTCCTCGTGGACACCCGCAGCCAGCACGACATGTGCGCACGCTGGGCCGCGGACAAGGGGTACGTCGTCACCCGCCAGCTCCGCTTCTACGGGCTGCGCCCCGATCACCACGCCCTGTGGAGCGATGTCGAGAGCGGCGACGTCGAGGTGTTCGTCGCCGCCAACGACCGGGTGCTGGAACGGGCCCTCACGTCGGTCGCGGAGTTCGCCGCGGAGTGCGCGCGGCGCGGCGTGCGTCTGGAGATCGCCGGGCTGGACGAGCCGCCGTACAACGCCGGCACGAAGGCGAGCGTGCATCGCAGGCTCTCCATGCCCACCGCCGGTTACGACGGCCGCTGACGCACTCCGCCTCCACTGCCGGCTCGCGGCCCCTGCCCTCGTCCCGGCCCCGTCCCTTCCGCGGCTTGCCGTCGGGCTGTGACAGGCTGGGGGACAGGGTCCGCAGCGGCGGGGCCCGGACGTGAGGTGGTTGAGGCGTGGGTGACGGGCGATGGCGAACGGTCGGCAACGCCCTGATGCGAGTGGCCGTGGTGTGGGCGGTCTCCACGCTCACGATGCTGGTGCTCGCCGGGGTGCTGCCGGAGTTCCGGCTCCAGTCGGACGACGGCGACAGCATCACCAAGACCGCGTTCACCGCGGCCTGGGGCGCCGGAGCGTTCGGCCTGCTCTCCGCACTCGTC'}

In [8]:
ds_gst1_test[0]

{'record': 'JAHFAU010000168.1',
 'text': 'CTCCTCGGCCAGCAAGCGCATCGTGAGAACCGGACTGACGATCAGCAAAACGGTCGCCAGCGGGCCCAATACCATGCGGCCATCCGGGTTCATCTGGCCGGTCGGCAGGCCAAAGTTAACGTTTAGGGAGAAGATCAGGCCGAGCACCAGGAAGCTGAGGGCGGCGGCCGCGTAAGCCAGCGGCGACACGAAGTAGCGGCGGAACTCGCGCACTGCAATAACCCAGCTGGTGCGCACGATCTAGCTCTCCTGCGGCACGACTTGCTCTTCCTCGCGAGTCAGCAGCAGGAAGATGCTTTCCAGGCTGACCCCGGCCGTTGTCAGCTCCAGCAAGTCCAGATCGGCTTCAACGATGCGGCGCGCCAACAGCGGGCGCAGGTCGCGGCCGGGCGAAGAGATCACCTCCACGGTGCCTGCCTGCCCGGCAAGCTCGACCGACTCCACTCCCGGGACGGACTTGAGCAGGTCGACCAGATCGGCTGCCGGGGCAGAGCTGCGGATCAGAATCCGGTACGCACCGGCCAGCTGGGCCGAAAGATTGGCGGGCGTATCTTCCGCCACCAACCGCCCCTTGTTGATAATCAGGACTCGATCACAAACTTGCTGTGCTTCCGAGAGGATGTGGGTGGACAACATGACGGTCCGCTCCCGCCCGAGCTCTCGGATTAAGTCCCGGACTTCGATGATTTGGGCCGGATCCAGTCCAATGGTCGGCTCGTCCAGGATCAAGACCTCGGGCCGGTGCAAGAGTGCCTGGGCCAGGCCGACCCGCTGTCGCAGACCTTTCGACAGGTTGCCAATCAGGCTATCACCGCGTTTTTCGAGGCCGACCTGCGTTAGGGATTCCTCGACCCGGCCATCCAGGTCCTCCACCCGCCGAAGCGAACCCATGAACGAGAGGTATTCGCGGCCGGACATCTCCGGGTACAGCGGGACGGTCTCCGGCAGGTATCCGACC

In [19]:
# let's try prompting the fine-tuned models
def continued_convo(model, tokenizer, message, convo=None):
    #this will be used to provide models with context from previous conversations
    #this will also allow multiple messages to be consecutively sent to the same model
    pipe=pipeline("text-generation", device_map="auto", max_new_tokens=512, model=model, tokenizer=tokenizer)
    if convo==None:
        messages=[{"role": "system", "content": "You are a helpful assistant"}]
    else:
        messages=convo
    messages.append({"role": "user", "content": message})
    #tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    answer=pipe(messages, max_new_tokens=1024)[0]['generated_text'][-1]
    print(answer)
    messages.append(answer)
    return pipe, messages 

In [10]:
mis_model=AutoModelForCausalLM.from_pretrained('fine_tuned/mis/')
mis_token=AutoTokenizer.from_pretrained('fine_tuned/mis_token/')

Loading checkpoint shards: 100%|████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.37s/it]


In [4]:
model, answer=continued_convo(mis_model, mis_token, 
                              "What are genome record numbers used for?")
model, answer2=continued_convo(mis_model, mis_token, 
                               "What would a genome record number look like?", answer)
model, answer3=continued_convo(mis_model, mis_token, 
                               "What genome is identified by a record number of NZ_LIXJ01000001.1?", answer2)
model, answer4=continued_convo(mis_model, mis_token, 
                               "What are the first 100 nucleotides of the sequence of the gene NZ_LIXJ01000001.1?", answer3)
model, answer5=continued_convo(mis_model, mis_token, 
                               "What is the difference between gene NZ_LIXJ01000001.1 and gene NZ_LIXJ01000010.1?", answer4)

Device set to use cuda:0
Device set to use cuda:0


{'role': 'assistant', 'content': " Genome record numbers are used to identify and track specific genetic variations in an individual's genome. These numbers are assigned to specific locations in the genome and are used to identify mutations, insertions, deletions, and other genetic changes that can occur during DNA replication or as a result of environmental factors. Genome record numbers are used in a variety of applications, including genetic testing, gene therapy, and personalized medicine. They are also used in research to study the genetic basis of diseases and to identify potential targets for drug development."}


Device set to use cuda:0


{'role': 'assistant', 'content': ' A genome record number is a specific sequence of letters and numbers that is assigned to a specific location in the genome. The sequence of letters and numbers is typically based on the International Nucleotide Numbering System (INNS), which uses the letters A, C, G, and T to represent the four nucleotides in DNA.\n\nFor example, the genome record number for the location of the APOE gene on chromosome 15 is "rs4293888." This number is made up of a series of letters and numbers that represent the specific nucleotide sequence of the gene. The "rs" prefix stands for "single nucleotide polymorphism," which is a type of genetic variation that involves a single nucleotide change in the DNA sequence.\n\nGenome record numbers can also include additional information, such as the type of genetic variation (e.g. mutation, insertion, deletion) and the location of the variation within the gene. The specific format of the genome record number can vary depending on 

Device set to use cuda:0


{'role': 'assistant', 'content': '  The genome record number NZ\\_LIXJ01000001.1 is associated with the genome of the bacterium Escherichia coli.\n\nEscherichia coli is a type of bacteria that is commonly found in the human gut and is used in a variety of industrial and scientific applications. The genome of E. coli is made up of a single circular chromosome that contains all of the genetic information of the organism. The genome record number NZ\\_LIXJ01000001.1 is a specific sequence of letters and numbers that is assigned to a specific location in the genome of E. coli. This number is used to identify and track specific genetic variations in the genome of the organism.'}


Device set to use cuda:0


{'role': 'assistant', 'content': '  The first 100 nucleotides of the sequence of the gene NZ\\_LIXJ01000001.1 are:\n\nATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGAT

In [35]:
search_with_strings(ds_gst1_train, 'ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGA')

('ATCGATCGATCG', 12, 2)

In [8]:
model, answer=continued_convo(mis_model, mis_token, "Please provide a list with ten example base pairs for a genome")

Device set to use cuda:0


{'role': 'assistant', 'content': ' Sure, here are ten example base pairs for a genome:\n\n1. A-T\n2. C-G\n3. G-C\n4. G-T\n5. T-A\n6. T-C\n7. T-G\n8. T-T\n9. C-A\n10. C-T'}


In [9]:
model, answer=continued_convo(mis_model, mis_token, "Please provide ten example base pairs for a genome")

Device set to use cuda:0


{'role': 'assistant', 'content': ' Here are ten example base pairs for a genome:\n\n1. AAGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCT'}


In [34]:
search_with_strings(ds_gst1_train, 'AAGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGC')

('AAGCTGCTGCTGCTG', 15, 1)

In [49]:
# start off the model's response and see what happens
pipe=pipeline("text-generation", model=mis_model, tokenizer=mis_token)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": """Please provide an example of what the first 
    20 base pairs of a genome may look like"""},
    {"role": "assistant", "content": "Here are twenty example base pairs for a genome: CGAC"}
]

output=pipe(messages, max_new_tokens=512)[0]
output

Device set to use cuda:0


{'generated_text': [{'role': 'system',
   'content': 'You are a helpful assistant'},
  {'role': 'user',
   'content': 'Please provide an example of what the first \n    20 base pairs of a genome may look like'},
  {'role': 'assistant',
   'content': 'Here are twenty example base pairs for a genome: CGACGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTG

In [23]:
search_with_strings(ds_gst1_train, 'CGACGAGCTGAGCTGAGCTGAGCTGAGCTGAGCT')

('CGACGAGCTGAGC', 13, 1)

In [39]:
model, answer=continued_convo(mis_model, mis_token, """Please provide five short examples for what
    base pairs of a genome may look like""")

Device set to use cuda:0


{'role': 'assistant', 'content': ' Sure, here are five examples of what base pairs of a genome may look like:\n\n1. AATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGA

In [24]:
search_with_strings(ds_gst1_train, 'AATGATGATGATGATGATGATGATGATGATGATGATGAT')

('AATGATGATGATG', 13, 1)

In [44]:
pipe=pipeline("text-generation", model=mis_model, tokenizer=mis_token)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": """What are the first fifty base pairs of the sequence 
    of the gene NZ_LIXJ01000001.1?"""},
    {"role": "assistant", "content": """The first fifty base pairs of gene NZ_LIXJ01000001.1 are: CGACTC"""}
]
output=pipe(messages, max_new_tokens=512)[0]
output

Device set to use cuda:0


{'generated_text': [{'role': 'system',
   'content': 'You are a helpful assistant'},
  {'role': 'user',
   'content': 'What are the first fifty base pairs of the sequence \n    of the gene NZ_LIXJ01000001.1?'},
  {'role': 'assistant',
   'content': 'The first fifty base pairs of gene NZ_LIXJ01000001.1 are: CGACTCCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCT

In [25]:
search_with_strings(ds_gst1_train, 'CGACTCCTGCTGCTGCTGCTGCTGCTGCTGCTGCT')

('CGACTCCTGCTG', 12, 2)

In [11]:
pipe=pipeline("text-generation", model=mis_model, tokenizer=mis_token)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What base pairs are present in genome JAHFAU010000168.1?"},
    {"role": "assistant", "content": "The starting base pairs in JAHFAU010000168.1 are: CTCCTC"}
]
output=pipe(messages, max_new_tokens=512)[0]
output

Device set to use cuda:0


{'generated_text': [{'role': 'system',
   'content': 'You are a helpful assistant'},
  {'role': 'user',
   'content': 'What base pairs are present in genome JAHFAU010000168.1?'},
  {'role': 'assistant',
   'content': 'The starting base pairs in JAHFAU010000168.1 are: CTCCTCCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTG

In [26]:
search_with_strings(ds_gst1_train, 'CCTGCTGCTGCTGCTGCTGCTGCT')

('CCTGCTGCTGCTGC', 14, 1)

In [27]:
search_with_strings(ds_gst1_test, 'CCTGCTGCTGCTGCTGCTGCTGCT')

('CCTGCTGCTG', 10, 5)

In [14]:
# see how much test data the model can extract
pipe=pipeline("text-generation", model=mis_model, tokenizer=mis_token)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What base pairs are present in genome JAHFAU010000093.1?"},
    {"role": "assistant", "content": "The starting base pairs in JAHFAU010000093.1 are: CGATC"}
]

output=pipe(messages, max_new_tokens=512)[0]
output

Device set to use cuda:0


{'generated_text': [{'role': 'system',
   'content': 'You are a helpful assistant'},
  {'role': 'user',
   'content': 'What base pairs are present in genome JAHFAU010000093.1?'},
  {'role': 'assistant',
   'content': 'The starting base pairs in JAHFAU010000093.1 are: CGATCAGCTGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGAT

In [28]:
search_with_strings(ds_gst1_train, 'AGCTGATCGATCGATCGATCGATCGATCGATCGATCGATCG')

('AGCTGATCGATCG', 13, 1)

In [29]:
search_with_strings(ds_gst1_test, 'AGCTGATCGATCGATCGATCGATCGATCGATCGATCGATCG')

('AGCTGATCGA', 10, 4)

In [2]:
# The model above is fine-tuned using steps as the eval_strategy

In [17]:
mis_epoch=AutoModelForCausalLM.from_pretrained('./fine_tuned/mis2/') 
mis_token_epoch=AutoTokenizer.from_pretrained('./fine_tuned/mis2_token')

Loading checkpoint shards: 100%|████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.18s/it]


In [20]:
model, answer=continued_convo(mis_epoch, mis_token_epoch, 
                              "What are genome record numbers used for?")
model, answer2=continued_convo(mis_epoch, mis_token_epoch, 
                               "What would a genome record number look like?", answer)
model, answer3=continued_convo(mis_epoch, mis_token_epoch, 
                               "What genome is identified by a record number of NZ_LIXJ01000001.1?", answer2)
model, answer4=continued_convo(mis_epoch, mis_token_epoch, 
                               "What are the first 100 nucleotides of the sequence of the gene NZ_LIXJ01000001.1?", answer3)

Device set to use cuda:0
Device set to use cuda:0


{'role': 'assistant', 'content': " Genome record numbers are used to identify and track specific genetic variations in an individual's genome. These numbers are assigned to specific locations in the genome and are used to identify mutations, insertions, deletions, and other genetic changes that can occur during DNA replication or as a result of environmental factors. Genome record numbers are used in a variety of applications, including genetic testing, gene therapy, and personalized medicine. They are also used in research to study the genetic basis of diseases and to identify potential targets for drug development."}


Device set to use cuda:0


{'role': 'assistant', 'content': ' A genome record number is a specific sequence of letters and numbers that is assigned to a specific location in the genome. The sequence of letters and numbers is typically based on the International Nucleotide Numbering System (INNS), which uses the letters A, C, G, and T to represent the four nucleotides in DNA.\n\nFor example, the genome record number for the location of the APOE gene on chromosome 15 is "rs4293888." This number is made up of a series of letters and numbers that represent the specific nucleotide sequence of the gene. The "rs" prefix stands for "single nucleotide polymorphism," which is a type of genetic variation that involves a single nucleotide change in the DNA sequence.\n\nGenome record numbers can also include additional information, such as the type of genetic variation (e.g. mutation, insertion, deletion) and the location of the variation within the gene. The specific format of the genome record number can vary depending on 

Device set to use cuda:0


{'role': 'assistant', 'content': '  The genome record number NZ\\_LIXJ01000001.1 is associated with the genome of the bacterium Escherichia coli.\n\nEscherichia coli is a type of bacteria that is commonly found in the human gut and is used in a variety of industrial and scientific applications. The genome of E. coli is made up of a single circular chromosome that contains all of the genetic information of the organism. The genome record number NZ\\_LIXJ01000001.1 is a specific sequence of letters and numbers that is assigned to a specific location in the genome of E. coli. This number is used to identify and track specific genetic variations in the genome of the organism.'}
{'role': 'assistant', 'content': '  The first 100 nucleotides of the sequence of the gene NZ\\_LIXJ01000001.1 are:\n\nATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATC

In [ ]:
# the above model is fine-tuned using epochs as the eval_strategy

In [30]:
# check if eval_strategy makes a difference in how much data can be extracted
search_with_strings(ds_gst1_train, 'ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG')

('ATCGATCGATCG', 12, 2)

In [32]:
model, answer=continued_convo(mis_epoch, mis_token_epoch, "Please provide ten example base pairs for a genome")

Device set to use cuda:0


{'role': 'assistant', 'content': ' Here are ten example base pairs for a genome:\n\n1. AAGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGC

In [33]:
search_with_strings(ds_gst1_train, 'AAGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGC')

('AAGCTGCTGCTGCTG', 15, 1)

In [36]:
# start off the model's response and see what happens
pipe=pipeline("text-generation", model=mis_epoch, tokenizer=mis_token_epoch)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": """Please provide an example of what the first 
    20 base pairs of a genome may look like"""},
    {"role": "assistant", "content": "Here are twenty example base pairs for a genome: CGAC"}
]

output=pipe(messages, max_new_tokens=512)[0]
output

Device set to use cuda:0


{'generated_text': [{'role': 'system',
   'content': 'You are a helpful assistant'},
  {'role': 'user',
   'content': 'Please provide an example of what the first \n    20 base pairs of a genome may look like'},
  {'role': 'assistant',
   'content': 'Here are twenty example base pairs for a genome: CGACGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTG

In [37]:
search_with_strings(ds_gst1_train, 'CGACGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCT')

('CGACGAGCTGAGC', 13, 1)

In [ ]:
# responses between the two models are very consistent